### Import code and external modules

In [3]:
from datetime import date, timedelta
import yfinance as yf

from LSTM.lstmPredictor import LSTMPredictor
from LSTM.pfTrainer import pfLSTMTrainer

from CovMat.optimizer import WeightOptimizer
from CovMat.cov_matrix import MPTCovMat
from CovMat.JSONReader import read_json
from CovMat.backtesting import BackTester



### Set parameters

In [1]:
train_range = tuple(date(2020,1,1), date(2020,12,31)) 
test_range = tuple(date(2021,1,1), date(2021,12,31)) #used to train lstm
eval_range= tuple(date(2022,1,1), date(2022,12,31)) #to evaluate PF performance

#Target (risk free)
rf = 0.027
rf_period = timedelta(120)

#Portfolio goals
alpha = 0.2 #calmar
beta = 0.3 #omega
gamma = 0.5 #sortino

lr_lstm = 1e-3
lr_alloc = 1e-2
epochs = 10 #training epochs for lstm
num_iter = 10000 #weight updates for allocator

tickers_json_path = 'LSTM/omxs30.json' #path where json file with ticker names is located
comp_ticker = '^OMX'

NameError: name 'train_start' is not defined

### Initialize objects

In [ ]:
ticker_list = read_json(tickers_json_path)
tickers = yf.Tickers(ticker_list)
comp_df = yf.Ticker(comp_ticker).history(start=test_range[0],end=test_range[1],auto_adjust=True,repair=True)

In [ ]:
allocator = WeightOptimizer(num_iter=num_iter,lr=lr_alloc,num_assets=len(ticker_list),risk_free=rf,risk_free_period=rf_period)
lstm_model = LSTMPredictor(len(ticker_list),300,len(ticker_list),n_layers=3,)
lstm_trainer = pfLSTMTrainer(lstm_model,allocator,tickers,train_range,test_range,lr_lstm)

### Optimize weights

In [ ]:
for epoch in epochs:
    lstm_trainer.optimize_weights(alpha,beta,gamma)
    lstm_trainer.train_lstm(comp_df=comp_df)

trained_lstm = lstm_trainer.model